

{% include toc title="In This Lesson" icon="file-text" %}



<div class='notice--success' markdown="1">



## <i class="fa fa-graduation-cap" aria-hidden="true"></i> Learning Objectives



After completing this tutorial, you will be able to:



* Effectively classify a raster dataset using classes determined by exploring a histogram of the data.



## <i class="fa fa-check-square-o fa-2" aria-hidden="true"></i> What you need



You will need a computer with internet access to complete this lesson and the data for week 4 of the course.



[<i class="fa fa-download" aria-hidden="true"></i> Download Week 4 Data (~500 MB)](https://ndownloader.figshare.com/files/7525363){:data-proofer-ignore='' .btn }



</div>



We can break our data analysis workflow into several steps as follows:



* **Data Processing:** load and "clean" the data. This may include cropping, dealing with NA values, etc

* **Data Exploration:** understand the range and distribution of values in your data. This may involve plotting histograms scatter plots, etc

* **More Data Processing & Analysis:** This may include the final data processing steps that you determined based upon the data exploration phase.

* **Final Data Analysis:** The final steps of your analysis - often performed using information gathered in the early data processing / exploration stages of your workflow.

* **Presentation:** Refining your results into a final plot or set of plots that are cleaned up, labeled, etc.



Please note - working with data is not a linear process. There are no defined

steps. As you work with data more, you will develop your own workflow and approach.




In [1]:
```{r setup}

# load libraries

library(raster)

library(rgdal)

library(ggplot2)



# set working directory

setwd("~/Documents/earth-analytics")

options(stringsAsFactors = F)

```




Note: try mapview() is a function that allows you to create interactive maps of

spatial data using leaflet as a base.



https://cran.r-project.org/web/packages/mapview/index.html




In [1]:
```{r load-data}

# load data

pre_dtm <- raster("data/week3/BLDR_LeeHill/pre-flood/lidar/pre_DTM.tif")

pre_dsm <- raster("data/week3/BLDR_LeeHill/pre-flood/lidar/pre_DSM.tif")



post_dtm <- raster("data/week3/BLDR_LeeHill/post-flood/lidar/post_DTM.tif")

post_dsm <- raster("data/week3/BLDR_LeeHill/post-flood/lidar/post_DSM.tif")



# import crop extent

crop_ext <- readOGR("data/week3/BLDR_LeeHill", "clip-extent")



```




Calculate the difference.




In [1]:
```{r diff-data, fig.cap="difference between pre and post flood dtm." }

# calculate dtm difference

dtm_diff_uncropped <- post_dtm - pre_dtm

plot(dtm_diff_uncropped)

```




Next, crop the data.




In [1]:
```{r crop-data, fig.cap="cropped data"}

# crop the data

dtm_diff <- crop(dtm_diff_uncropped, crop_ext)

plot(dtm_diff,

     main="cropped data")



# get a quick glimpse at some of the values for a particular "row"

# note there are a LOT of values in this raster so this won't print all values.

# below i used the head() function to limit the n umber of values returned to 6.

# that way a lot of numbers don't print out in my final knitr output.

head(getValues(dtm_diff, row = 5))



# view max data values

dtm_diff@data@max

dtm_diff@data@min

```


In [1]:
```{r plot-histogram, fig.cap="initial histogram"}

# plot histogram of data

hist(dtm_diff,

     main="Distribution of raster cell values in the DTM difference data",

     xlab="Height (m)", ylab="Number of Pixels",

     col="springgreen")

```


In [1]:
```{r plot-histogram-xlim, fig.cap="initial histogram w xlim to zoom in"}

hist(dtm_diff,

     xlim=c(-2,2),

     main="Histogram of pre-post flood DTM differences \nZoomed in to -2 to 2 on the x axis",

     col="brown")



# see how R is breaking up the data

histinfo <- hist(dtm_diff)

histinfo



# how many breaks does R use in the default histogram

length(histinfo$breaks)



# summarize values in the data

summary(dtm_diff, na.rm=T)



```




## Breaks



Above, we saw that we can see how R breaks up our data to create a histogram.

R, by default, creates 35 bins to plot a histogram of our raster data. We can increase the number of breaks or bins that the hist0gram uses with the

argument:



`breaks=number`



In the example below, I used a very large number - 500 so we can see the bins.




In [1]:
```{r plot-histogram-breaks, fig.cap="initial histogram w xlim to zoom in and breaks" }

# where are most of our values?

hist(dtm_diff,

     xlim=c(-2,2),

     breaks=500,

     main="Histogram \n Zoomed in to -2-2 on the x axis w more breaks",

     xlab="Height (m)", ylab="Number of Pixels",

     col="springgreen")

```




### Histogram with custom breaks



We can create custom breaks or bins in a histogram too. To do this, we pass the same `breaks`

argument a vector of numbers that represent the range for each bin in our histogram.






In [1]:
```{r plot-histogram-breaks2, fig.cap="histogram w custom breaks"}

# We may want to explore breaks in our histogram before plotting our data

hist(dtm_diff,

     breaks=c(-20, -10, -3, -.3, .3, 3, 10, 50),

     main="Histogram with custom breaks",

     xlab="Height (m)" , ylab="Number of Pixels",

     col="springgreen")



```




Finally, let's plot the data using the breaks that we created for our histogram

above. We know that there is a high number of cells with a value between -1 and 1.

So let's consider that when we select the colors for our plot.




In [1]:
```{r plot-data, fig.cap="Plot difference dtm."}

# plot dtm difference with breaks

plot(dtm_diff,

     breaks=c(-20, -10, -3, -.3, .3, 3, 10, 50),

     col=terrain.colors(7),

     main="DTM Difference \n Using manual breaks")

```




## Custom plot colors



Next, let's adjust the colors that we use to plot our raster. to do that we will

create a vector of colors, each or which will represent one of our numeric "bins"

of raster values.



This mimics a classified map - we are still exploring our data!




In [1]:
```{r set-colors}

# how many breaks do we have?

# NOTE: we will have one less break than the length of this vector

length(c(-20,-10,-3,-1, 1, 3, 10, 50))

```




Set number of colors based upon how many breaks or bins we have in our data

above we have 8 numbers in our breaks vector. this translates to 7 bins each or which requires a unique color.




In [1]:
```{r plot-with-unique-colors, fig.cap="Plot difference dtm with custom colors."}

# create a vector of colors - one for each "bin" of raster cells

diff_colors <- c("palevioletred4", "palevioletred2", "palevioletred1", "ivory1",

                "seagreen1","seagreen2","seagreen4")

plot(dtm_diff,

     breaks=c(-20, -3, -.3, .3, 3, 50),

     col=diff_colors,

     legend=F,

     main="Plot of DTM differences\n custom colors & manual breaks")



# make sure legend plots outside of the plot area

par(xpd=T)

# add the legend to the plot

legend(x=dtm_diff@extent@xmax, y=dtm_diff@extent@ymax, # legend location

       legend=c("-20 to -3", "-3 to -.3",

                "-.3 to .3", ".3 to 3", "3 to 50"),

       fill=diff_colors,

       bty="n",

       cex=.7)



```




## Crop and replot



We can zoom into a part of the raster manually - by first cropping the data

using a manually created plot extent. Then plotting the newly cropped raster subset.




In [1]:
```{r create-new-extent, fig.cap="cropped dtm subset" }



# new_extent <- drawExtent()

new_extent <- extent(473690, 474155.2, 4434849, 4435204)

new_extent



# crop the raster to a smaller area

dtm_diff_crop <- crop(dtm_diff, new_extent)



# Plot the cropped raster

plot(dtm_diff_crop,

     breaks=c(-20, -3, -.3, .3, 3, 50),

     col=diff_colors,

     legend=F,

     main="Lidar DTM Difference \n cropped subset")



# grab the upper right hand corner coordinates to place the legend.

legendx <- dtm_diff_crop@extent@xmax

legendy <- dtm_diff_crop@extent@ymax



par(xpd=TRUE)

legend(legendx, legendy,

       legend=c("-20 to -3", "-3 to -.3", "-.3 to .3",

                ".3 to 3", "3 to 50"),

       fill=diff_colors,

       bty="n",

       cex=.8)



```


In [1]:
```{r clear-par-settings }

dev.off()

```




## Create a final classified dataset



When we have decided what break points work best for our data, then we may chose

to classify the data.




In [1]:
```{r classify-data }



# create reclass vector

reclass_vector <- c(-20,-3, -2,

                    -3, -.5, -1,

                    -.5, .5, 0,

                    .5, 3, 1,

                    3, 50, 2)



reclass_matrix <- matrix(reclass_vector,

                         ncol=3,

                         byrow = T)



reclass_matrix



```




## Reclassify difference raster




In [1]:
```{r reclassify-raster-diff, fig.cap="final plot" }



diff_dtm_rcl <- reclassify(dtm_diff, reclass_matrix)



plot(diff_dtm_rcl,

     col=diff_colors,

     legend=F,

     main="Reclassified, Cropped Difference DTM \n difference in meters")

par(xpd=T)

legend(dtm_diff@extent@xmax, dtm_diff@extent@ymax,

       legend=c("-20 to -10", "-10 to -3", "-3 to -.5",

                "-.5 to .5", "1 to 3", "3 to 10", "10 to 50"),

       fill=diff_colors,

       bty="n",

       cex=.8)

```




Finally view the final histogram




In [1]:
```{r histogram-of-diff-rcl, fig.cap="histogram of differences"}

hist(diff_dtm_rcl,

     main="Histogram of reclassified data",

     xlab="Height Class (meters)",

     ylab="Number of Pixels")



```






The above histogram looks odd. This is because we are trying to force our discrete

data into bins. A barplot is a more appropriate plot.




In [1]:
```{r barplot-of-diff-rcl, fig.cap="histogram of differences"}

barplot(diff_dtm_rcl,

     main="Barplot of reclassified data",

     xlab="Height Class (meters)",

     ylab="Frequency of Pixels",

     col=diff_colors)



```








Now let's look at one last thing. What would the distribution look like if

we set all values between -.5 to .5 to NA?




In [1]:
```{r reclass, fig.cap="histogram of final cleaned data"}

# create a new raster object

diff_dtm_rcl_na <- diff_dtm_rcl

# assign values between -.5 and .5 to NA

diff_dtm_rcl_na[diff_dtm_rcl_na >= -.5 & diff_dtm_rcl_na <= .5] <- NA

# view histogram

barplot(diff_dtm_rcl_na,

     main="Histogram of data \n values between -.5 and .5 set to NA",

     xlab="Difference Class",

     col=diff_colors)



# view summary of data

summary(diff_dtm_rcl_na)

```
